In [31]:
import pandas as pd
df_loc = pd.read_csv('Locations.csv')
df_prod = pd.read_csv('Pizza_Production.csv')
df_loc['store_id'] = df_loc['extended_store_id'].str[:4].astype(int)
print(df_loc.head())

print(df_loc.shape)

      city          location extended_store_id  store_id
0   Boston          Back Bay           3205-01      3205
1  Chicago          Evanston           4153-00      4153
2  Chicago  Magnificent Mile           3860-00      3860
3  Chicago      Wrigleyville           5431-02      5431
4   Denver   Golden Triangle           5019-00      5019
(7, 4)


In [32]:
print(df_prod.head())
print(df_prod.shape)

  work_date   work_week  store_id                 metric  quantity
0  1/2/2015  12/28/2014      5019        Pizzas Produced        29
1  1/2/2015  12/28/2014      5019           Hours Worked        98
2  1/9/2015    1/4/2015      3860  Cheesy Bread Produced        34
3  1/9/2015    1/4/2015      3860        Pizzas Produced        77
4  1/5/2015    1/4/2015      3860           Hours Worked       113
(6351, 5)


In [199]:
#a)  Merge files together to get the city friendly name in the dataframe
#b: i)  Merge files together to get the city friendly name in the dataframe

df_comb = (pd.merge(df_loc, df_prod, left_on='store_id', right_on='store_id', how='left'))
print(df_comb.shape)
df_comb.head()

(6351, 8)


,city,location,extended_store_id,store_id,work_date,work_week,metric,quantity
0,Boston,Back Bay,3205-01,3205,1/23/2015,1/18/2015,Pizzas Produced,26
1,Boston,Back Bay,3205-01,3205,1/21/2015,1/18/2015,Hours Worked,84
2,Boston,Back Bay,3205-01,3205,1/22/2015,1/18/2015,Hours Worked,94
3,Boston,Back Bay,3205-01,3205,1/23/2015,1/18/2015,Hours Worked,100
4,Boston,Back Bay,3205-01,3205,1/20/2015,1/18/2015,Hours Worked,97


In [200]:
df_comb['work_date']= pd.to_datetime(df_comb['work_date'])
df_comb['month'] = df_comb.work_date.dt.month.astype(int)
df_comb['year'] = df_comb.work_date.dt.year.astype(int)
print(df_comb.head())

     city  location extended_store_id  store_id  work_date  work_week  \
0  Boston  Back Bay           3205-01      3205 2015-01-23  1/18/2015   
1  Boston  Back Bay           3205-01      3205 2015-01-21  1/18/2015   
2  Boston  Back Bay           3205-01      3205 2015-01-22  1/18/2015   
3  Boston  Back Bay           3205-01      3205 2015-01-23  1/18/2015   
4  Boston  Back Bay           3205-01      3205 2015-01-20  1/18/2015   

            metric  quantity  month  year  
0  Pizzas Produced        26      1  2015  
1     Hours Worked        84      1  2015  
2     Hours Worked        94      1  2015  
3     Hours Worked       100      1  2015  
4     Hours Worked        97      1  2015  


In [201]:
df_comb_raw = df_comb
df_comb = df_comb[(df_comb['metric'] != 'Cheesy Bread Produced')]
df_comb_raw.shape

(6351, 10)

In [202]:
v = df_comb.groupby(['city','location','year','month','metric'])['quantity'].sum()
v.to_csv('PPH.csv')

In [193]:
#b: ii)  Pizza produced per hour worked

pph = pd.read_csv('PPH.csv')
last_row = pph.shape[0]
for i in pph.index:
    if i < (last_row-2):
        if (pph["city"][i] == pph["city"][i+1]) & (pph["location"][i] == pph["location"][i+1]) & (pph["year"][i] == pph["year"][i+1]) & (pph["month"][i] == pph["month"][i+1]):
            pph["metric"][i] = pph["quantity"][i+1] / pph["quantity"][i]

pph_filtered = pph.loc[pph["metric"] != 'Pizzas Produced']
del pph_filtered["quantity"]

pd.crosstab([pph_filtered["city"], pph_filtered["location"]], [pph_filtered["year"], pph_filtered["month"]], values=pph_filtered['metric'], aggfunc='max')

C:\Users\singh\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


year                              2015                                \
month                               1         2         3         4    
city        location                                                   
Boston      Back Bay           0.27472  0.411887  0.592573  0.755659   
Chicago     Evanston          0.415061  0.407449  0.331334   0.64711   
            Magnificent Mile  0.326758  0.207317  0.282587  0.250288   
            Wrigleyville           0.5  0.396081  0.371783  0.296795   
Denver      Golden Triangle   0.125443  0.206147  0.163275   0.11507   
Los Angeles Airport           0.116834  0.153543  0.386004  0.206398   
Tampa       Ybor City         0.539768  0.326294  0.768549  0.775141   

year                                                                    \
month                               5         6          7          8    
city        location                                                     
Boston      Back Bay          0.546688  0.267956   0.249448   0.259439   
Chicago     Evanston          0.457388  0.134321   0.306093   0.278257   
            Magnificent Mile  0.269248  0.208484   0.129845    0.18692   
            Wrigleyville      0.317612  0.307073   0.225207   0.245213   
Denver      Golden Triangle   0.161994  0.175064  0.0560386  0.0488082   
Los Angeles Airport           0.134385   0.21481   0.225597   0.137077   
Tampa       Ybor City         0.502172  0.246146   0.283394   0.184623   

year                                              ...      2016             \
month                               9         10  ...        3          4    
city        location                              ...                        
Boston      Back Bay          0.291627  0.456351  ...  0.313347   0.334586   
Chicago     Evanston          0.261783  0.383531  ...  0.558333   0.546861   
            Magnificent Mile  0.291906  0.235376  ...  0.348152   0.397371   
            Wrigleyville      0.156189  0.415197  ...  0.290643  0.0643564   
Denver      Golden Triangle   0.144828  0.265137  ...   0.12703   0.194673   
Los Angeles Airport           0.190107  0.239171  ...  0.317214   0.383701   
Tampa       Ybor City         0.107452  0.192596  ...  0.418153   0.348963   

year                                                                   \
month                               5         6         7          8    
city        location                                                    
Boston      Back Bay          0.138249  0.345727  0.517532   0.417957   
Chicago     Evanston          0.443795  0.433289  0.510877   0.278293   
            Magnificent Mile  0.399022  0.355065  0.527921  0.0984285   
            Wrigleyville           NaN       NaN       NaN        NaN   
Denver      Golden Triangle   0.146898  0.185817  0.115894   0.138511   
Los Angeles Airport           0.323518  0.412201  0.458173   0.417317   
Tampa       Ybor City         0.229563  0.309013  0.291805   0.266667   

year                                                                      
month                               9         10        11            12  
city        location                                                      
Boston      Back Bay          0.415571  0.344342  0.216294      0.166165  
Chicago     Evanston          0.591562  0.527394  0.228936      0.471924  
            Magnificent Mile  0.303377  0.207758  0.282996      0.237205  
            Wrigleyville           NaN       NaN       NaN           NaN  
Denver      Golden Triangle   0.139279  0.225788  0.159332      0.224917  
Los Angeles Airport           0.377056  0.404965  0.572627      0.579832  
Tampa       Ybor City         0.231678  0.244517  0.291753  Hours Worked  

[7 rows x 24 columns]

In [286]:
df_comb_raw['day'] = df_comb_raw.work_date.dt.day.astype(int)
df_comb_raw = df_comb_raw[(df_comb_raw['metric'] != 'Hours Worked')]


dff = df_comb_raw.groupby(['city', 'year','month', 'day','metric'])['quantity'].sum()
dff.to_csv('cheesy.csv')

In [284]:
#C:  Cheesy bread coupon campaign.  Please see comments/recommendation in the main pdf file.

cheesy = pd.read_csv('cheesy.csv')
last_row = cheesy.shape[0]
for i in cheesy.index:
    if i < (last_row-2):
        if (cheesy["city"][i] == cheesy["city"][i+1]) & (cheesy["year"][i] == cheesy["year"][i+1]) & (cheesy["month"][i] == cheesy["month"][i+1]) & (cheesy["day"][i] == cheesy["day"][i+1]):
            cheesy["metric"][i] = cheesy["quantity"][i] / (cheesy["quantity"][i+1])



cheesy_filtered = cheesy_filtered[(cheesy_filtered['metric'] != 'Pizzas Produced')]
cheesy_filtered = cheesy_filtered[(cheesy_filtered['metric'] != 'Cheesy Bread Produced')]
print(cheesy_filtered.sort_values(['metric'], ascending=[False]))


C:\Users\singh\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


         city  year  month  day     metric
1172  Chicago  2016      5   31        4.5
1081  Chicago  2016      4    7    3.51515
1101  Chicago  2016      4   20    3.22222
1183  Chicago  2016      6    8    3.18519
1065  Chicago  2016      3   29    2.94286
...       ...   ...    ...  ...        ...
1210  Chicago  2016      6   26  0.0797546
1375  Chicago  2016     10   20  0.0773481
842   Chicago  2015     11    1  0.0695187
468   Chicago  2015      1    2  0.0691057
811   Chicago  2015     10    9  0.0613208

[811 rows x 5 columns]
